In [60]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import date,datetime,timedelta
# from math import floor
# import matplotlib.pyplot as plt
# from tqdm.notebook import tqdm

from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy, TrailingStrategy
from backtesting.test import SMA

In [56]:
def stock_data_all(ticker):

    start_date1 = date.today()-timedelta(days=5000)

    start_date = start_date1.strftime("%Y-%m-%d")

    # Request data:
    data = yf.download(ticker,
                        interval="1d",
                        start = start_date)
    
    return data
data1 = stock_data_all('SPY')

[*********************100%***********************]  1 of 1 completed


In [57]:
data = data1[['Open','High','Low','Adj Close','Volume']]
data = data.rename(columns={'Adj Close': 'Close'}, inplace=False)
data = data.dropna()


In [58]:
class SmaCross(SignalStrategy,
               TrailingStrategy):
    n1 = 50
    n2 = 150
    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize the parent classes
        super().init()
        
        # Precompute the two moving averages
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0)  # Upwards/long only
        
        # Use 95% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * .95
                
        # Set order entry sizes using the method provided by 
        # `SignalStrategy`. See the docs.
        self.set_signal(entry_size=entry_size)
        
        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        self.set_trailing_sl(2)

In [59]:
bt = Backtest(data, SmaCross,
              cash=100000, commission=.002,
              exclusive_orders=True)

output = bt.run()
bt.plot()